In [ ]:
import numpy as np
import sys
sys.path.append('../../build')
import libry as ry
import time

In [ ]:
C = ry.Config()
C.addFile("myRobot.g")
D = C.view()
q0 = np.array([45, 80, -80])/180*np.pi
C.setJointState(q0)

C.frame("link3").setColor([1,1,1,.7])

test = C.addFrame("test")
test.setShape(ry.ST.sphere, [0.1])
test.setColor([1, 0, 0])
test.setPosition([0,0,1])

In [ ]:
for _ in range(10):
    q = np.random.rand(3)
    C.setJointState(q)

    s1, s12, s123 = np.sin(q[0]), np.sin(q[0]+q[1]), np.sin(q[0]+q[1]+q[2])
    c1, c12, c123 = np.cos(q[0]), np.cos(q[0]+q[1]), np.cos(q[0]+q[1]+q[2])
    
    pos_computed = np.array([
        s1 + s12 + s123,
        0,
        1 + c1 + c12 + c123
    ])
    test.setPosition(pos_computed)
    J_computed = np.array([
        [c1 + c12 + c123, c12 + c123, c123],
        [0, 0, 0],
        [-s1 - s12 - s123, -s12 - s123, - s123]
    ])
    
    y, J = C.evalFeature(ry.FS.position, ["end_effector"])
    
    diff_phi = pos_computed - y
    print('phi error: ', np.abs(diff_phi).max())
    
    diff_J = J_computed - J
    print('J error: ', np.abs(diff_J).max())
    input()
    
    

# Generating motions with feature and pseudoinverse

In [ ]:
import numpy as np
import sys
sys.path.append('../../build')
import libry as ry
import time

In [ ]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()
q0 = C.getJointState()

In [ ]:
obj = C.addFrame("object")
# set frame parameters, associate a shape to the frame, 
pos_obj = np.array([.5,0,1.5])
pos_obj0 = pos_obj.copy()
obj.setPosition(pos_obj)
obj.setQuaternion([1,0,0,0])
obj.setShape(ry.ST.sphere, [.02])
obj.setColor([1,0,0])

## Tracking an object

Suppose the feature $\phi$ is a mapping to the position difference between the gripper and the object (case 1 above). If this feature space is endowed with a stable dynamics, e.g., $$\dot{y}=-y,$$ corresponding joint motions will let the gripper position to approach to the object (i.e., the position difference converges to 0). The following code exactly does that.



In [ ]:
tau = .01
obj.setPosition(pos_obj)
C.setJointState(q0)
q = C.getJointState()
for t in range(1000):
    time.sleep(0.01)
    
    pos_obj[0] = 1 - np.sin(0.01*t) #move the obj for fun!
    obj.setPosition(pos_obj)
    
    [y,J] = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "object"])
    vel_ee = -y

#     vel = np.linalg.pinv(J) @ vel_ee; #less stable!
    vel = np.linalg.inv(J.T@J + 1e-2*np.eye(q.shape[0])) @ J.T @ vel_ee;

    q += tau*vel
    C.setJointState(q) #set your robot model to match the real q

In [ ]:
tau = .01
C.setJointState(q0)
obj.setPosition(pos_obj0)
q = C.getJointState()
for t in range(1000):
    time.sleep(0.01)
    pos_obj[0] = 1 - np.sin(0.01*t) #move the obj for fun!
    obj.setPosition(pos_obj)
    
    #evaluate a first feature
    [y1,J1] = C.evalFeature(ry.FS.positionRel, ["R_gripperCenter", "object"])
    #you can multiply y1 and J1 here with some number, to adjust the importance of the first feature
    c1 = 0.5
    y1, J1 = c1*y1, c1*J1
    
    #evaluate a second feature
    [y2,J2] = C.evalFeature(ry.FS.scalarProductXY, ["R_gripperCenter","world"])
    c2 = 2
    y2, J2 = c2*y2, c2*J2

    # Third "qItself" feature is for regularization (small joint velocity)
    vel_ee = np.block([-y1, c2-y2, np.zeros(q.shape[0])]) 
    J = np.block([[J1],[J2],[1e-1*np.eye(q.shape[0])]])
    
    
    ## these are equivalent!
#     vel =  np.linalg.pinv(J) @ vel_ee;
    vel =  np.linalg.inv(J.T@J) @ J.T @ vel_ee;


    q += tau*vel
    C.setJointState(q) 

## Using Komo for IK

In [ ]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

In [ ]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

In [ ]:
obj = C.addFrame("object")
obj.setPosition([1., 0., 1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

In [ ]:
IK = C.komo_IK(False)
# IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.0]);

In [ ]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

In [ ]:
C.setFrameState( IK.getFrameState(0) )

In [ ]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# redefine the IK problem
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.2])

IK.addObjective([1.], 
                ry.FS.positionDiff, 
                ["R_gripperCenter", "L_gripperCenter"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.0]);
# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )